In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications as apps
import pandas as pd
import kagglehub
from pathlib import Path
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import train_test_split
import gc


2025-04-09 11:40:21.808558: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744195221.875996   40196 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744195221.894614   40196 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744195222.019678   40196 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744195222.019711   40196 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744195222.019714   40196 computation_placer.cc:177] computation placer alr

In [2]:
def set_memory_growth():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
      try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
      except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

def set_memory_limit(memory_limit):
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit)]
        )

    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

set_memory_limit(4096)
#set_memory_growth()
keras.mixed_precision.set_global_policy("mixed_float16")

pd.set_option('display.max_columns', None)

1 Physical GPU, 1 Logical GPUs


I0000 00:00:1744195228.126748   40196 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4096 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
SEED = 42
BATCH_SIZE_TOTAL = 128
BATCH_SIZE = 8
GRADIENT_ACCUMULATION_STEPS = int(BATCH_SIZE_TOTAL / BATCH_SIZE)
PATIENCE = 5

saved_models_dir = Path("../saved_models")
saved_models_dir.mkdir(parents=True, exist_ok=True)
histories_dir = Path("../histories")
histories_dir.mkdir(parents=True, exist_ok=True)
logs_dir = Path("../logs")
logs_dir.mkdir(parents=True, exist_ok=True)

# Create Datasets

In [4]:
# Download latest version of data
# Use the next 2 lines if not downloaded before
image_dir = kagglehub.dataset_download("bloox2/fieldplant")
image_dir = Path(image_dir) / "train"

# Use the next lines of code if your data has been downloaded already, but you are offline.  Will used cached data.
# image_dir = "~/.cache/kagglehub/datasets/bloox2/fieldplant/versions/1/train"
# print("Path to dataset files:", image_dir)

# Create Models

In [5]:
def get_model_and_preprocessing(app_name):
    app = getattr(keras.applications, app_name)
    model_name = dir(app)[0]
    model = getattr(app, model_name)
    input_shape = model().input_shape[1:]
    model = model(include_top=False, input_shape=input_shape, weights="imagenet")
    model.trainable = False
    preprocessing = getattr(app, "preprocess_input")
    return model, preprocessing

In [6]:
# def get_pre_classifier_layers(model_name):
#     if "vgg" in model_name:
#         return keras.Sequential([
#             keras.layers.GlobalAveragePooling2D(),
#             keras.layers.Dense(1024, activation="relu"),
#             keras.layers.Dense(1024, activation="relu") ])
#     else:
#         return keras.Sequential([keras.layers.GlobalAveragePooling2D()])

    

In [7]:
def get_pre_classifier_layers(model_name):
    dropout_value = 0.2
    
    if 'vgg' in model_name:
        return keras.Sequential([
            keras.layers.GlobalAveragePooling2D(),
            keras.layers.Dense(4096, activation='relu'),
            keras.layers.Dense(1072, activation='relu'),
            keras.layers.Dropout(dropout_value)
        ])
    else:
        seq = keras.Sequential()
        seq.add(keras.layers.GlobalAveragePooling2D())
        if "mobilenet" not in model_name:
            seq.add(keras.layers.Dense(1024, activation = 'relu'))
        seq.add(keras.layers.Dropout(dropout_value))
        return seq

In [8]:
def build_model(app_name, activation, num_classes):
    model, preprocessing = get_model_and_preprocessing(app_name)
    pre_classifier_layers = get_pre_classifier_layers(model.name)

    inputs = keras.Input(shape=model.input_shape[1:])
    x = preprocessing(inputs)
    x = model(x, training=False)
    x = pre_classifier_layers(x)
    outputs = keras.layers.Dense(num_classes, activation=activation, name="classifier_layer")(x)

    model_name = model.name
    model = keras.Model(inputs, outputs, name=model_name)

    return model

In [9]:
def get_hyperparameters(methodology):
    multilabel = methodology == "multilabel"
    
    methodologies = ["multiclass", "multilabel"]
    losses = ["categorical_crossentropy", "binary_crossentropy"]
    activation = ["softmax", "sigmoid"]
    metrics = ["categorical_accuracy", "binary_accuracy"]

    idx = methodologies.index(methodology)

    metrics = [metrics[idx]]
    f1_score_weighted = keras.metrics.F1Score(average="weighted", threshold=0.5 if multilabel else None, name="f1_score_weighted", dtype=None)
    f1_score_per_class = keras.metrics.F1Score(average=None, threshold=0.5 if multilabel else None, name="f1_score_per_class", dtype=None)
    metrics.append(f1_score_weighted)
    metrics.append(f1_score_per_class)
        
    hyperparams = [losses[idx], activation[idx], metrics]
    
    return hyperparams

In [10]:
def print_model_info(model):
    print()
    print("print_model_info() start")
    compile_config = model._compile_config.config
    optimizer = compile_config['optimizer'].get_config()
    classifier_activation = model.get_layer(name="classifier_layer").activation.__name__

    print("Model name:", model.name)
    print("Input shape:", model.input_shape)
    print("Optimizer name:", optimizer['name'], "learning_rate:", np.round(optimizer['learning_rate'], 6))
    print("Loss:", compile_config['loss'])
    print("Metrics:")
    for metric in compile_config['metrics']:
        print(metric if isinstance(metric, str) else metric.get_config())
    print("Classifier layer activation function:", classifier_activation)
    print()
    print(model.summary(expand_nested=True, show_trainable=True))
    print()

    print("print_model_info() end")
    print()


In [11]:
def get_callbacks(best_epoch=None, model_name=None, fine_tuning=False):
    if fine_tuning:
        baseline_loss = best_epoch.val_loss.values[0]
        print("Previous best epoch (starting counting from 0):", best_epoch.epoch.values[0])
        print("Previous best epoch (starting counting from 1, as per training loop):", best_epoch.epoch.values[0] +1)

        print("Baseline val_loss:", baseline_loss)
    cbs = [
        keras.callbacks.EarlyStopping(
            patience=PATIENCE, restore_best_weights=True,
            baseline=None if not fine_tuning else baseline_loss,
            verbose=1),
        keras.callbacks.ModelCheckpoint(
            filepath=f"{saved_models_dir}/{model_name}.keras",
            save_best_only=True, monitor="val_loss",
            verbose=1,
            initial_value_threshold=None if not fine_tuning else baseline_loss
        )]
    return cbs


# GET DATASETS

In [12]:
def get_dataframe(filtered=False, sample=False):
    filename = "filtered" if filtered else "unfiltered"
    all_csv_files = list(Path("../data").glob("*"))
    csv_file = [csv for csv in all_csv_files if filename in csv.name][0]
    df = pd.read_csv(csv_file)
    print()

    if sample:
        print("Using sampled DF")
        df = df.sample(frac=0.2)
        df = df.loc[(df!=0).any(axis=1)]
    print("Df shape:", df.shape)
    print()
    return df   

In [13]:
def get_train_test_splits(df, filtered=False, test_size=0.2):
    col_names = list(df.columns)
    split_fn = get_stratified_splits if filtered else get_nonstratified_splits
    (X_train, X_test, X_val, y_train, y_test, y_val) = split_fn(df, test_size=test_size)
    train_df = pd.merge(X_train, y_train, left_index=True, right_index=True)
    test_df = pd.merge(X_test, y_test, left_index=True, right_index=True)
    val_df = pd.merge(X_val, y_val, left_index=True, right_index=True)
    
    train_df.columns = col_names
    test_df.columns = col_names
    val_df.columns = col_names

    return train_df, test_df, val_df
    
def get_stratified_splits(df, test_size=0.2):
    columns = list(df.columns)
    X = df.filename.to_frame().to_numpy()
    y = df.drop(columns=["filename"]).to_numpy()

    X_train, y_train, X_test_val, y_test_val = iterative_train_test_split(X, y, test_size=0.2)
    X_test, y_test, X_val, y_val = iterative_train_test_split(X_test_val, y_test_val, test_size=0.5)
    datasets = (X_train, X_test, X_val, y_train, y_test, y_val)
    datasets = [pd.DataFrame(dataset) for dataset in datasets]
    return tuple(datasets)

def get_nonstratified_splits(df, test_size=0.2):
    X = df.filename
    y = df.drop(columns=["filename"])
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=SEED)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=SEED)

    return (X_train, X_test, X_val, y_train, y_test, y_val)

In [14]:
def decode_img(filename, img_size):
    filepath = str(image_dir) + "/" + filename
    img = tf.io.read_file(filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    return img
    
def process_dataset(filename, labels, img_size):
    img = decode_img(filename, img_size=img_size)
    return img, labels

def configure_datasets_for_performance(datasets, shuffle=False, batch_size=BATCH_SIZE):
    configured_datasets = []
    ds_sizes = [int(ds.cardinality().numpy()) for ds in datasets]
    print("Ds sizes:", ds_sizes)
    print("NP argmax:", np.argmax(ds_sizes))
    for i, dataset in enumerate(datasets):
        if int(dataset.cardinality().numpy()) == ds_sizes[np.argmax(ds_sizes)]:
            print(f"Shuffling dataset {i}")
            print()
            dataset = dataset.shuffle(buffer_size=dataset.cardinality(), reshuffle_each_iteration=True)
        dataset = dataset.batch(batch_size=batch_size, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.cache()
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        configured_datasets.append(dataset)
    return tuple(configured_datasets)

In [15]:
def datasets_from_dataframes(img_size, splits=None):
    datasets = []
    ds_names = ["train", "val", "test"]
    for i, split in enumerate(splits):
        img = split.filename
        labels = split.drop(columns=["filename"])
        
        dataset = tf.data.Dataset.from_tensor_slices((img, labels))
        print(ds_names[i], "length:", len(dataset))

        dataset = dataset.map(lambda x,y: process_dataset(x,y, img_size))
        print(ds_names[i], dataset.element_spec)
        print()
        
        datasets.append(dataset)
    return tuple(datasets)

In [16]:
def get_datasets(img_size, df, test_size=0.2):
    splits = get_train_test_splits(df, test_size=test_size)
    datasets = datasets_from_dataframes(img_size, splits=splits)
    return datasets

# CREATE MODELS

In [17]:
def freeze_model(model):
    print()
    print("freeze_model() start")

    base_layer_name = ""
    num = 0
    if "mobilenet" in model.name:
        base_layer_name = "mobilenetv2_1.00_224"
        num = 120
    elif "vgg16" in model.name:
        base_layer_name = "vgg16"
        num = 14
    elif "inception_v3" in model.name:
        base_layer_name = "inception_v3"
        num = 172
    elif "resnet" in model.name:
        base_layer_name = "inception_resnet_v2"
        num = 516

    print(f"Freeze layers up to layer {num}")
    for layer in model.get_layer(name=base_layer_name).layers[:num]:
        layer.trainable = False
    for layer in model.get_layer(name=base_layer_name).layers[num:]:
        layer.trainable = True

    print("freeze_model() end")
    print()

In [18]:
def train_model(best_epoch=None, model=None, train_ds=None, val_ds=None, epochs=100, loss=None, metrics=[], fine_tuning=False):
    print()
    print("train_model() start")

    string_suffix = "FT" if fine_tuning else "CLF"
    print("Training -", string_suffix)

    if fine_tuning:
        freeze_model(model)
        
    compile_model(model, loss=loss, metrics=metrics, fine_tuning=fine_tuning)
    print_model_info(model)
       
    cbs = get_callbacks(model_name=model.name, fine_tuning=fine_tuning, best_epoch=best_epoch)

    # +1 to best epoch so that it matches the number given in the training loop.  (So best=7 becomes best=8)
    # +1 again so that it begins training from the epoch following the previous best. (So inital_epoch=8 becomes inital_epoch=9)
    initial_epoch = 0 if not fine_tuning else best_epoch.epoch.values[0] + 1 + 1

    print(f"Starting training at epoch {initial_epoch}. (starting counting from 0, as per df)" )
    print(f"Starting training at epoch {initial_epoch + 1}. (starting counting from 1, as per training loop)" )
    print()

    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=cbs, initial_epoch=initial_epoch)
    history_df = pd.DataFrame(history.history)
    history_df['model'] = model.name
    history_df['epoch'] = history.epoch
    history_df['type'] = string_suffix
    
    save_filename = f"{str(histories_dir)}/{model.name}_{string_suffix}.csv"
    print()
    print("Saving file as:", save_filename)
    history_df.to_csv(save_filename, index=False)  
    
    best_epoch = history_df.loc[history_df.val_loss == history_df.val_loss.min()]
    print(f"Best epoch number (starting from 0, as per df): {best_epoch.epoch.values[0]}")
    print(f"Best epoch number (starting from 1, as per training loop): {best_epoch.epoch.values[0] + 1}")
    print()


    display(history_df)

    print()
    print("train_model() end")

    return best_epoch
    

In [19]:
def compile_model(model, loss=None, metrics=[], fine_tuning=False):
    print()
    print("compile_model() start")
    opt = keras.optimizers.Adam
    lr = (float(opt().learning_rate) / 10) if fine_tuning else float(opt().learning_rate)
    
    model.compile(
        loss=loss,
        optimizer=keras.optimizers.Adam(learning_rate=lr, gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS),
        metrics=metrics
    )
    print("compile_model() end")
    print()

In [20]:
def run_all(epochs=5, sample=True, model_name=None):

    print()
    print("run_all() start")

    i = 0
    num_models = 0

    if model_name == "all":
        app_names = ["mobilenet_v2", "vgg16", "inception_v3", "inception_resnet_v2"]
    else:
        app_names = [model_name]
    methodologies = ["multiclass", "multilabel"]
    filter_options = [False, True]
    

    # # # Use for smaller combinations
    # app_names = ["vgg16"]
    # methodologies = ["multiclass", "multilabel"]
    # filter_options = [False, True]


    # Get total count of models to be training
    for filtered in filter_options:
        for methodology in methodologies:
            if not filtered and (methodology == "multilabel"):
                continue
            for app_name in app_names:
                num_models += 1

    # Training Loop
    for filtered in filter_options:
        for methodology in methodologies:
            if not filtered and (methodology == "multilabel"):
                continue
                
            loss, activation, metrics = get_hyperparameters(methodology)
    
            df = get_dataframe(filtered=filtered, sample=sample)
            num_classes = len(df.columns[1:])
            
            for app_name in app_names:
                i += 1
                model = build_model(app_name, activation=activation, num_classes=num_classes)
                model.name = model.name + "_" + ("filtered" if filtered else "unfiltered") + "_" + methodology
                img_size = model.input_shape[1:3]
                datasets = get_datasets(img_size, df=df)
                train_ds, test_ds, val_ds = configure_datasets_for_performance(datasets)

                  # Train top Classifier
                for fine_tuning in [False, True]:
                    print()
                    print(f"Model: {i} of {num_models}")
                    print("app_name:", app_name)
                    print("filtered:", filtered)
                    print("methodology:", methodology)
                    
                    best_epoch = train_model(model=model, best_epoch=None if not fine_tuning else best_epoch,
                                             train_ds=train_ds, val_ds=val_ds,
                                             epochs=epochs, loss=loss, metrics=metrics,
                                             fine_tuning=fine_tuning)

                                   
                # Clear memory
                del model
                gc.collect()
                tf.keras.backend.clear_session()

                print(); print(); print(); print();
                print("BEGINNING NEXT MODEL IF EXISTS...")

                # # Train top Classifier
                # best_epoch = train_model(model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, loss=loss, metrics=metrics, fine_tuning=False)
                # # Fine Tuning
                # train_model(best_epoch=best_epoch, model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, loss=loss, metrics=metrics, fine_tuning=True)

                
    print("run_all() end")
    print()

# RUN THE PROGRAMME
The cell below runs the entire programme ddesigned above.<br>
Change the parameters below as needed.

epochs:  Choose num epochs, however EarlyStopping will likely stop before this.<br>
sample: Choose whether to use a sample subset of the data.<br>
print_to_file (not necessary): Choose whether to print the cells output to a file in /logs_dir, or print to cell output on screen.  If printing to file, you will not be able to see the cell output on screen as the programme runs, and so cannot track progress.<br><br>

The previous cell function run_all(), can be altered to run all combinations of models, methodologies, and filtered options.  Or can run with any combination of these variables as needed.

In [21]:
from IPython.utils.capture import capture_output

epochs = 100
sample = False
print_to_file = True

model_name = "all"
# "mobilenet_v2", "vgg16", "inception_v3", "inception_resnet_v2"
#model_name = "mobilenet_v2"


if print_to_file:
    with capture_output() as captured_output:
        model = run_all(epochs=epochs, sample=sample, model_name=model_name)
else:
    model = run_all(epochs=epochs, sample=sample, model_name=model_name)


I0000 00:00:1744195239.952031   40283 service.cc:152] XLA service 0x7f7f840516e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744195239.952070   40283 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-04-09 11:40:40.051251: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744195240.593916   40283 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1744195247.192404   40283 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-04-09 11:40:50.620712: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1389', 444 bytes spill stores, 464 bytes spill loads

2025-04-09 11:42:35.1469

In [23]:
save_log_filename = f"{logs_dir}/{model_name}.txt"
print("Saving log to:", save_log_filename)
with open(save_log_filename, "w") as f:
    f.write(captured_output.stdout)

Saving log to: ../logs/mobilenet_v2.txt
